In [73]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [74]:
# Read csv file in data folder
filename = os.listdir(path='data')[0]

In [75]:
# Create dataframe
df_financial_data = pd.read_csv(f'data/{filename}', index_col=False)

print(df_financial_data.head())

  Account Type Account Number Transaction Date  Cheque Number  \
0     Chequing  01542-5018304        5/16/2024            NaN   
1     Chequing  01542-5018304        5/16/2024            NaN   
2     Chequing  01542-5018304        5/16/2024            NaN   
3     Chequing  01542-5018304        5/16/2024            NaN   
4     Chequing  01542-5018304        5/17/2024            NaN   

     Description 1         Description 2     CAD$  USD$  
0  STAFF - PAYROLL                   NaN  1627.10   NaN  
1       Email Trfs      E-TRANSFER SENT    -20.00   NaN  
2       Email Trfs      E-TRANSFER SENT   -171.50   NaN  
3     E-TRF REQFUL                   NaN  -450.00   NaN  
4       Email Trfs  E-TRANSFER RECEIVED     23.95   NaN  


In [76]:
# Remove credit card payments
df_financial_data = df_financial_data[df_financial_data['Description 1'] != 'PAYMENT - THANK YOU / PAIEMENT - MERCI']

# Make all blank values into NULL
df_financial_data = df_financial_data.replace(r'^\s*$', None, regex=True)

# Trim all strings
df_financial_data = df_financial_data.map(lambda x: x.strip() if isinstance(x, str) else x)

# Rename columns
cols = ['ACCOUNT_TYPE','ACCOUNT_NUMBER','TRANSACTION_DATE','CHEQUE_NUMBER','DESCRIPTION_ONE','DESCRIPTION_TWO','CAD','USD']
df_financial_data.columns = cols

print(df_financial_data.head())

  ACCOUNT_TYPE ACCOUNT_NUMBER TRANSACTION_DATE  CHEQUE_NUMBER  \
0     Chequing  01542-5018304        5/16/2024            NaN   
1     Chequing  01542-5018304        5/16/2024            NaN   
2     Chequing  01542-5018304        5/16/2024            NaN   
3     Chequing  01542-5018304        5/16/2024            NaN   
4     Chequing  01542-5018304        5/17/2024            NaN   

   DESCRIPTION_ONE      DESCRIPTION_TWO      CAD  USD  
0  STAFF - PAYROLL                  NaN  1627.10  NaN  
1       Email Trfs      E-TRANSFER SENT   -20.00  NaN  
2       Email Trfs      E-TRANSFER SENT  -171.50  NaN  
3     E-TRF REQFUL                  NaN  -450.00  NaN  
4       Email Trfs  E-TRANSFER RECEIVED    23.95  NaN  


In [77]:
max_length = df_financial_data['DESCRIPTION_TWO'].str.len().max()
print(max_length)

32.0


In [78]:
load_dotenv('secrets.env')

# SQL Connection
uid = 'sa'
pwd = os.getenv('SQL_SERVER_PWD')
server = os.getenv('DATABASE_URL')
database = 'FINANCIAL_TRANSACTIONS'
driver = 'ODBC+Driver+17+for+SQL+Server'

SQL_conn_str = f'mssql+pyodbc://{uid}:{pwd}@{server}/{database}?driver={driver}'
sql_conn = create_engine(SQL_conn_str)

In [79]:
# Export
df_financial_data.to_excel('Data Validation/transaction_data.xlsx', index=False)
df_financial_data.to_sql('RBC', sql_conn, schema='dbo', if_exists='append', index=False)

155

In [80]:
# Close SQL connection
sql_conn.dispose()